In [28]:
from flask import Flask, render_template, request
import numpy as np
import torch
import re
import base64
import Ipynb_importer
import cnn
import heapq
from torchvision import transforms
from PIL import Image
from torch.autograd import Variable
from datetime import timedelta
print("torch version : {}".format(torch.__version__))

torch version : 1.4.0


In [29]:
app = Flask(__name__)
app.send_file_max_age_default = timedelta(seconds = 1)

model_file = './model.pth'
model = torch.load(model_file)

In [30]:
@app.route('/')
def index():
    return render_template("index.html")

In [31]:
@app.route('/predict/', methods = ['Get', 'POST'])
def process():
    parseImage(request.get_data())
    img = to_transform()
    response = predict(img)

    return response

In [32]:
def parseImage(imgData):
    imgStr = re.search(b'base64,(.*)', imgData).group(1)
#     print(imgStr)
    with open('./output.png', 'wb') as output:
        output.write(base64.decodebytes(imgStr))
    

In [33]:
def to_transform():
    img = Image.open('output.png')
    df_transforms = transforms.Compose(
        [transforms.Resize((28, 28)), transforms.Grayscale(num_output_channels = 1), transforms.ToTensor()])
    copy_img = df_transforms(img)
    copy_img = copy_img.unsqueeze(0)
    return copy_img

In [34]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim = 0)
    return X_exp / partition 

In [35]:
def predict(img):
    if torch.cuda.is_available:
        img = img.cuda()
    out = model(img)
    _, pred = torch.max(out, 1)
    print(pred)
    out = out.squeeze(0)
    lst = softmax(out.detach()).cpu().numpy()
    h1, h2, h3 = heapq.nlargest(3,range(len(lst)), lst.take)
    str1 = '{:.2%}'.format(lst[h1])
    str2 = '{:.2%}'.format(lst[h2])
    str3 = '{:.2%}'.format(lst[h3])
    
    resp = pred[0].cpu().numpy()
    return '{},{},{},{},{},{},{}'.format(str(resp), str(h1), str1, str(h2), str2, str(h3), str3)


In [36]:
app.run(host = "127.0.0.1", port = 8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2020 15:13:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2020 15:13:55] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2020 15:13:55] "GET /static/jquery-3.2.0.min.js HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2020 15:13:55] "GET /static/index.js HTTP/1.1" 304 -


tensor([1], device='cuda:0')


127.0.0.1 - - [31/May/2020 15:13:59] "POST /predict/ HTTP/1.1" 200 -


tensor([2], device='cuda:0')


...

127.0.0.1 - - [31/May/2020 15:19:02] "POST /predict/ HTTP/1.1" 200 -


tensor([8], device='cuda:0')


127.0.0.1 - - [31/May/2020 15:19:04] "POST /predict/ HTTP/1.1" 200 -


tensor([9], device='cuda:0')


127.0.0.1 - - [31/May/2020 15:19:07] "POST /predict/ HTTP/1.1" 200 -
